# **도커-복습-3**

**도커 이미지**

**도커 이미지 생성**

```bash
docker run -it --name commit_test ubuntu:22.04

 컨테이너 내부에서 
echo test_first! >> first
```

- docker commit [OPTIONS]  container [REPOSITORY[:TAG]]

```bash
docker commit -a "alicek106" -m "my first commit" commit_test commit_test:first

# -a 옵션은 author 
```

**이미지 구조 이해**

- 이미지를 커밋할 때 컨테이너에서 변경된 사항만 새로운 레이어로 저장한다.

- 이미지의 레이어 구조는 `docker history` 명령을 통해 좀 더 쉽게 확인할 수 있다.

```bash
docker history commit_test:first
```

- 삭제되는 이미지의 부모 이미지가 존재하지 않아야만 해당 이미지의 파일이 실제로 삭제된다.

**이미지 추출**

- `docker save` 명령어를 사용해서 파일로 추출 또는 로드 가능

```bash
docker save -o ubuntu_22_04.tar ubuntu:22.04

docker load -i ubuntu_22_04.tar
```

- docker save & docker load : docker image를 tar 파일로 추출하고 로드하는 역할로 이미지에 변경이 있지 않은 오리지널 이미지와 동일하다.
- docker export & docker import : docker container를 tar 파일로 추출하고 로드하는 역할로 오리지널 이미지를 아카이빙하여 `하나의 layer`로 저장된 이미지를 생성한다.
- **이미지를 단일 파일로 저장하는 것은 효율적인 방법이 아니다.**
    - 추출된 이미지는 레이어 구조의 파일이 아닌 단일 파일이기 때문에 여러 버전의 이미지를 추출하면 이미지 용량을 각기 차지한다.

**이미지 배포**

- 위에서 만든 이미지에 태그 달기
    - tag 명령어로 이미지의 이름을 변경했다고 해서 기존의 이름이 사라지는 것은 아니며, 같은 이미지를 가리키는 새로운 이름을 추가할 뿐이다.

```bash
docker tag commit_test:first bingbingpa/test-repo:0.0

# 배포
docker push bingbingpa/test-repo:0.0
```

- 저장소 webhook
    - 저장소에 이미지가 push 됐을 때 특정 URL 로 http 요청을 전송하도록 설정할 수 있다.

**도커 사설 레지스트리**

- 사설 레지스트리 컨테이너 생성
    - 도커 사설 레지스트리를 사용하면 개인 서버에 이미지를 저장할 수 있는 저장소를 만들 수 있다.
    - 이 레지스트리는 컨테이너로서 구현되므로 이에 해당하는 도커 이미지가 존재한다.
    
    ```bash
    docker run -d --name myregistry -p 5000:5000 registry:2.6
    
    # 정상 구동되었는데 레지스트리 컨테이너의 rest api call test
    curl localhost:5000/v2/
    ```
    
- 사설 레지스트리에 이미지 push 하기
    -
    ```bash
    docker tag commit_test:first 172.17.0.3:5000/my-image-name:0.0

    # https 관련 오류 발생한다. 
    # docker 옵션 변경해주기 
    sudo vim /etc/default/docker 

    # 이 옵션 추가하고 재시작
    DOCKER_OPTS="--insecure-registry=172.17.0.3:5000"
    ```